In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
# Set the maximum number of rows to display
pd.set_option('display.max_rows', None)


In [3]:
# URL to scrape
url = "https://www.basketball-reference.com/leagues/NBA_2025_games-march.html"
# Get the webpage content
response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', {'id': 'schedule'})
rows = table.find_all('tr')[1:]

game_data = []

for row in rows:
    cols = row.find_all('td')

    if cols:
        game_date = row.find('th').text.strip()
        away_team = cols[1].text.strip()
        home_team = cols[3].text.strip()
        away_score = cols[2].text.strip()
        home_score = cols[4].text.strip()
        if home_score:
            overtime = cols[6].text.strip() if cols[6].text.strip() else "n"
        else:
            overtime = ""

        game_data.append([game_date, away_team, home_team, away_score, home_score, overtime])
df = pd.DataFrame(game_data, columns=['Date', 'Away Team', 'Home Team', 'Away Score', 'Home Score', 'Overtime'])
df['Date'] = pd.to_datetime(df['Date'], format = "%a, %b %d, %Y").dt.date

In [4]:
import psycopg2
import sys
import os

In [5]:
connection = psycopg2.connect(
        host='192.168.4.64',       # e.g., 'localhost' or an IP address
        database='nba_odds', # e.g., 'mydatabase'
        user='gdshoda',   # e.g., 'postgres'
        password='122nd@NG' # e.g., 'mypassword'
    )

In [6]:
cursor = connection.cursor()

In [7]:
cursor.execute("SELECT MIN(date) FROM schedule WHERE home_points IS NULL")

In [8]:
min_date = cursor.fetchone()[0]

In [9]:
df = df.loc[(df['Date'] >= min_date) & (df['Home Score'] != '')]

In [10]:
for x, row in df.iterrows():
    home= row['Home Team']
    visitor = row['Away Team']
    date = row['Date']
    home_points = row['Home Score']
    away_points = row['Away Score']

    # SQL query to update the row where game_id and team match
    update_query =  """
                    UPDATE schedule
                    SET home_points = %s, away_points = %s
                    WHERE visitor = %s AND home = %s and date = %s;
                    """
    
    
    
    cursor.execute(update_query, (home_points, away_points, visitor, home, date))

    connection.commit()

    print(f"Updated Score for {visitor} @ {home} on {date}")
    break


Updated Score for Brooklyn Nets @ Cleveland Cavaliers on 2025-03-11
